In [ ]:
import numpy as np
import collections
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def brownian_bug(N, birth, death, step, A, phi, theta, df):
    bug_no = np.array(df['bug_no'].tolist(), float)
    pos_x = np.array(df['pos_x'].tolist(), float)
    pos_y = np.array(df['pos_y'].tolist(), float)
    
    counter = len(bug_no)
    temp_death = 0
    temp_birth = 0

    for i in range(counter):
        chance = np.random.random()
        if chance <= birth:
            bug_no = np.append(bug_no, max(bug_no) + 1)
            pos_x = np.append(pos_x, pos_x[i-temp_death])
            pos_y = np.append(pos_y, pos_y[i-temp_death])
            temp_birth += 1
        elif chance > birth and chance <= birth+death:
            bug_no = np.delete(bug_no, i-temp_death)
            pos_x = np.delete(pos_x, i-temp_death)
            pos_y = np.delete(pos_y, i-temp_death)
            temp_death += 1

    if len(bug_no) == 0:
        return print("Network: extinct")

    for k in range(len(pos_x)):
        angle = np.random.randint(0,361)
        angle = angle*np.pi/180

        pos_x[k] += step*np.cos(angle)
        pos_y[k] += step*np.sin(angle)

        #advection Stanford map
        pos_x[k] = pos_x[k] + ((A/(2*np.pi)) * np.sin(2*np.pi*pos_y[k]))
        pos_y[k] = pos_y[k] + pos_x[k]

        while pos_x[k] < 0 or pos_x[k] > 1 or pos_y[k] > 1 or pos_y[k] < 0:
            if pos_x[k] < 0: pos_x[k] +=1
            if pos_x[k] > 1: pos_x[k] -=1
            if pos_y[k] < 0: pos_y[k] +=1
            if pos_y[k] > 1: pos_y[k] -=1
        
    
    d = {'bug_no': bug_no, 'pos_x': pos_x, 'pos_y': pos_y}
    dataframe = pd.DataFrame(d)
    
    return dataframe

### Create network dataframe

In [ ]:
def distance(po_0,po_1, p1_0, p1_1):
    return np.sqrt((po_0 - p1_0)**2 + (po_1 - p1_1)**2)

def create_network_dataframe(dataframe):
    df = dataframe
    distances = 0
    transmission_radius = 0.1
    Network = []
    for i in range(len(df)):
        for j in range(len(df)):
            if i == j:
                continue
            else:
                distances = distance(df['pos_x'][i],df['pos_y'][i],df['pos_x'][j],df['pos_y'][j])
                if distances <= transmission_radius: 
                    edge = 1
                    dist = distances
                    cost = distances**2
                else: 
                    edge = 0
                    dist = distances
                    cost = 0

                Network.append({'1_node': i, 
                                '1_node_val_x': df['pos_x'][i],
                                '1_node_val_y': df['pos_y'][i],
                                '2_node': j, 
                                '2_node_val_x': df['pos_x'][j],
                                '2_node_val_y': df['pos_y'][j],
                                'edge': edge,
                                'cost': cost,
                                'distance': dist})

    df = pd.DataFrame(Network)
    return df

### Create network

In [ ]:
def create_network(dataframe):
    network = dataframe
    G_original_network = nx.Graph()

    n1 = network['1_node'].tolist()
    n2 = network['2_node'].tolist()
    nvalsx = network['1_node_val_x'].tolist()
    nvalsy = network['1_node_val_y'].tolist()
    nvalsx2 = network['2_node_val_x'].tolist()
    nvalsy2 = network['2_node_val_y'].tolist()
    edge = network['edge'].tolist()

    for i in range(len(n1)):
        G_original_network.add_node(int(n1[i]), pos = (nvalsx[i], nvalsy[i]))

    for i in range(len(n2)):
        G_original_network.add_node(int(n2[i]), pos = (nvalsx2[i], nvalsy2[i]))

    pos=nx.get_node_attributes(G_original_network,'pos')

    for i in range(len(n1)):
        if edge[i] != 0:
            G_original_network.add_edge(int(n1[i]),int(n2[i]),weight=float(edge[i]))
    
    return G_original_network, pos

### Visualize network

In [ ]:
def plot_network(G_original_network, pos):
    plt.figure(figsize=(3, 3))
    nx.draw_networkx(G_original_network, pos,with_labels=False,edge_color='blue',style = 'solid',width = 0.7, node_size = 8, node_color='blue')
    plt.draw()

### Remove node with largest degree and count the number of nodes of the largest subnetwork

In [ ]:
def Sort_Tuple(tup):  
    return(sorted(tup, key = lambda x: x[1], reverse = True))   

def targeted_attack(G, network):
    length_subgraphs = [len(component) for component in list(nx.connected_components(G))]
    maximum_size = np.max(length_subgraphs)
    
    node_degree = np.array(G.degree()) #(node, degree)
    node_degree = np.array(Sort_Tuple(node_degree)) #arrange from greatest to least
    remove_node = node_degree[0][0]
    
    network = network.drop(network.loc[network['1_node'] == remove_node].index).reset_index() 
    network = network.drop(network.loc[network['2_node'] == remove_node].index).reset_index()
    
    return network, remove_node, maximum_size

### Convert network dataframe to df dataframe

In [ ]:
def convert_network(network):
    node = network['1_node'].tolist()
    posx = network['1_node_val_x'].tolist()
    posy = network['1_node_val_y'].tolist()

    temp_df = []

    for i in range(len(node)):
        temp_df.append((node[i], posx[i], posy[i]))

    check_val = set()      #Check Flag
    temp = []
    for i in temp_df:
        if i[0] not in check_val:
            temp.append(i)
            check_val.add(i[0])

    temp = np.array(temp)
    if len(temp) == 0:
        return pd.DataFrame({})
    else:
        d = {'bug_no': temp[:,0], 'pos_x': temp[:,1], 'pos_y': temp[:,2]}
        dataframe = pd.DataFrame(d)
        return dataframe

### Initialize parameters and Brownian bugs

In [ ]:
def brownian_bug_network(N, time_cycle,stabilize, trial_number):
    np.random.seed()
    N = N
    time_cycle = time_cycle
    stabilize = stabilize
    birth = 0.25
    death = 0.25
    step =  1e-3
    A = 1
    phi, theta = 2*np.pi*np.random.random(2)

    bugs = []
    for i in range(N):
        bugs.append({'bug_no': i, 'pos_x': np.random.rand(), 'pos_y':np.random.rand()})

    df = pd.DataFrame(bugs)

    list_maximum_size_brownian = []
    number_disconnected_brownian = []
    list_assortativity = []

    list_maximum_size = []
    number_disconnected = []
    list_assortativity_not = []
    counter = 0

    for i in range(time_cycle):
        df = brownian_bug(N, birth, death, step, A, phi, theta, df)
        if i <= stabilize:
            df_not = df.copy()

        if i >= stabilize:
            #if counter == 0:
            #    print(df.head())
            #    print(df_not.head())

            network = create_network_dataframe(df)
            network_not = create_network_dataframe(df_not)
            if len(network) == 0:
                print('Network extinct')
                break 
            if len(network_not) == 0:
                print('Network extinct')
                break 


            G, pos =  create_network(network)
            G_not, pos_not =  create_network(network_not)


            network_dataframe, remove_node, maximum_size = targeted_attack(G, network)
            assortativity = nx.degree_assortativity_coefficient(G)
            network_dataframe_not, remove_node_not, maximum_size_not = targeted_attack(G_not, network_not)
            assortativity_not = nx.degree_assortativity_coefficient(G_not)
            

            df = convert_network(network_dataframe)
            df_not = convert_network(network_dataframe_not)

            if len(df) == 0:
                print('Network extinct')
                break
            if len(df_not) == 0:
                print('Network extinct')
                break

            #if i %10 == 0:
            print(i)
            #print('removed node:', remove_node)
            #print('size of largest component', maximum_size)
            #print('removed node:', remove_node_not)
            #print('size of largest component', maximum_size_not)

            list_maximum_size_brownian.append(maximum_size)
            list_assortativity.append(assortativity)
            number_disconnected_brownian.append(counter)

            list_maximum_size.append(maximum_size_not)
            list_assortativity_not.append(assortativity_not)
            number_disconnected.append(counter)

            counter += 1
    
    d = {'number_disconnected': number_disconnected, 'brownian': list_assortativity, 'not_brownian': list_assortativity_not}
    dataframe = pd.DataFrame(d)
    dataframe.to_csv('node_dis v. assortativity trials/trial_%s.csv'%(str(trial_number)), index=False)

In [ ]:
import multiprocessing

def simulate_wrapped(args):
    return simulate_trials(*args)

def simulate_trials(N, time_cycle, stabilize, trial_number):
    print('RUN : ', trial_number)
    dataframe = brownian_bug_network(N, time_cycle,stabilize, trial_number)
    dataframe.to_csv('trial_%s.csv'%(str(trial_number)), index=False)
    return dataframe.head()

trials = 1
max_trials = 7
N = 1000
time_cycle = 400
stabilize = 200

with multiprocessing.Pool() as workpool:
    data = workpool.map(simulate_wrapped, zip([N]*max_trials, [time_cycle]*max_trials, [stabilize]*max_trials, np.arange(trials, trials+max_trials, 1)))